## load JSON FHIR Questionnaires to SMART FHIR Server

In [170]:
#import relevant packages
from fhirclient import client
from fhirclient import server
import json
import pandas as pd
import numpy as np

import fhirclient.models.questionnaire as q
import fhirclient.models.patient as p
import fhirclient.models.humanname as hn
import fhirclient.models.questionnaireresponse as qr

#set up smart server
smart = server.FHIRServer(None, 'https://r4.smarthealthit.org')

In [171]:
#set up useful functions:

# pretty print
def pp(json_object):
    print(json.dumps(json_object, indent = 3))
    
def savetofile(json_object, filename):
    with open(filename, 'w') as outfile:
        json.dump(json_object, outfile)

In [172]:
#load mh_questionnaire json
with open('questionnaires/MHQuestionnaire_withValueSet.json', 'r') as h1:
    qjs1 = json.load(h1)
mh_questionnaire = q.Questionnaire(qjs1)

In [173]:
#load pa_questionnaire json
with open('questionnaires/PAQuestionnaire_withValueSet.json', 'r') as h2:
    qjs2 = json.load(h2)
pa_questionnaire = q.Questionnaire(qjs2)

In [174]:
#take loaded mental health json questionnaire and upload to smart server
mh_quest_load = q.Questionnaire.create(mh_questionnaire, smart)
mh_quest_id = mh_quest_load["id"]
print(mh_quest_id)

531841


In [175]:
#take loaded physical activity json questionnaire and upload to smart server
pa_quest_load = q.Questionnaire.create(pa_questionnaire, smart)
pa_quest_id = pa_quest_load["id"]
print(pa_quest_id)

531842


In [176]:
#pull mental health questionnaire from server and read to test:
mh_quest_download = q.Questionnaire.read(mh_quest_id, smart)

In [177]:
#pull physical activity questionnaire from server and read to test:
pa_quest_download = q.Questionnaire.read(pa_quest_id, smart)

In [178]:
#Add questionnaire responses for NHANES patients:
#read in csv with questionnaire response data:
pa_data = pd.read_csv('nhanes_data/activity_questionnaire.csv')
dpq_data = pd.read_csv('nhanes_data/depression_questionnaire.csv')

# r: pandas are generally not too useful to only iteration

In [179]:
# Verify: Questionnaire's linkId should match the linkIds in QuestionnaireResponse when created.
# Check if csv is the right one. 

# Physical Activity CSV and Questionnaire linkIds
pa_csv_questionids = sorted(list(pa_data.columns.values)[2:])
pa_questionnaire_linkids = sorted([item.linkId for item in pa_quest_download.item])
compare_df_pa = pd.DataFrame({'PA-CSV': pa_csv_questionids, 'PA-FHIR Questionnaire': pa_questionnaire_linkids})
print(compare_df_pa)

# Mental Health CSV and Questionnaire linkIds
mh_csv_questionids = sorted(list(dpq_data.columns.values)[2:])
mh_questionnaire_linkids = sorted([item.linkId for item in mh_quest_download.item])
compare_df_mh = pd.DataFrame({'PA-CSV': mh_csv_questionids, 'PA-FHIR Questionnaire': mh_questionnaire_linkids})
print(compare_df_mh)


    PA-CSV PA-FHIR Questionnaire
0   pad615                PAD615
1   pad630                PAD630
2   pad645                PAD645
3   pad660                PAD660
4   pad675                PAD675
5   pad680                PAD680
6   paq605                PAQ605
7   paq610                PAQ610
8   paq620                PAQ620
9   paq625                PAQ625
10  paq635                PAQ635
11  paq640                PAQ640
12  paq650                PAQ650
13  paq655                PAQ655
14  paq665                PAQ665
15  paq670                PAQ670
   PA-CSV PA-FHIR Questionnaire
0  dpq010                DPQ010
1  dpq020                DPQ020
2  dpq030                DPQ030
3  dpq040                DPQ040
4  dpq050                DPQ050
5  dpq060                DPQ060
6  dpq070                DPQ070
7  dpq080                DPQ080
8  dpq090                DPQ090
9  dpq100                DPQ100


In [222]:
#LOAD PHYSICAL ACTIVITY DATA INTO PY CLIENT MODEL:

pa_questionnaire_responses = []
looprange = range(0, len(pa_data))
for i in looprange:

    #assign variables to questionnaire response resource elements:
    #create py client questionnaire response
    pa_qr = qr.QuestionnaireResponse({'status': 'completed', 
                                      'questionnaire': f'Questionnaire/{pa_quest_id}',
                                      "identifier": {
                                        "system": "https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/PAQ_J.htm",
                                        "value": f'{str(int(pa_data.loc[i][1]))}'
                                    }})
    
    response_items = []

    #create questionnaire response items for each question, and item answers for each corresponding answer
    for question_id in pa_csv_questionids:
        qr_item = qr.QuestionnaireResponseItem()
        qr_item.linkId = question_id.upper()
        #answerItem:
        #check if Answer for this item is available, if not Skip, we do not add answerItem
        if str(pa_data.loc[i][question_id]) == 'nan':
            #skipped!
            continue 
            
        qr_item_answer = qr.QuestionnaireResponseItemAnswer()
        qr_item_answer.valueInteger = int(int(pa_data.loc[i][question_id]))
        
        qr_item.answer = [qr_item_answer]
        
        #add the question text to `qr_item.question`
        for i in range(0, len(pa_quest_download.item)):
            find_id = pa_quest_download.item[i].linkId
            if find_id == question_id:
                qr_item.text = pa_quest_download.item[i].text
        
        #add to response list:
        response_items.append(qr_item)
    
    pa_qr.item = response_items
    pa_questionnaire_responses.append(pa_qr)
    continue
    

In [161]:
#LOAD MENTAL HEALTH DATA INTO PY CLIENT MODEL:

mh_questionnaire_responses = []
looprange = range(0, len(dpq_data))
for i in looprange:
    
    #assign variables to questionnaire response resource elements:
    #create py client questionnaire response
    mh_qr = qr.QuestionnaireResponse({'status': 'completed', 'questionnaire': f'Questionnaire/{mh_quest_id}', 
                                      "identifier": {
                                            "system": "https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DPQ_J.htm",
                                            "value": f'{str(int(dpq_data.loc[i][1]))}'
                                        }})

    
    response_items = []

    #create questionnaire response items for each question, and item answers for each corresponding answer
    for question_id in mh_csv_questionids:
        qr_item = qr.QuestionnaireResponseItem()
        qr_item.linkId = question_id.upper()
        #answerItem:
        #check if Answer for this item is available, if not Skip, we do not add answerItem
        if str(dpq_data.loc[i][question_id]) == 'nan':
            #skipped!
            continue 
            
        qr_item_answer = qr.QuestionnaireResponseItemAnswer()
        qr_item_answer.valueInteger = int(int(dpq_data.loc[i][question_id]))
        
        qr_item.answer = [qr_item_answer]
        
        #TODO: Question title
        #Grab the question text for this given item from the Questionnaire (get item with the same `linkId`)
        #add the text to `qr_item.question` (https://www.hl7.org/fhir/questionnaireresponse-definitions.html#QuestionnaireResponse.item.text)
        
        #add to response list:
        response_items.append(qr_item)
        
    
    
    mh_qr.item = response_items
    mh_questionnaire_responses.append(mh_qr)
    continue
    

In [159]:
# Upload one physical activity questionnaire response to Server
#savetofile(pa_questionnaire_responses[0].as_json(), 'sample_qr.json')
pa_response_load = qr.QuestionnaireResponse.create(pa_questionnaire_responses[0], smart)
pa_response_id = pa_response_load["id"]
print(pa_response_id)


531839


In [162]:
# Upload one mental health questionnaire response to Server
#savetofile(mh_questionnaire_responses[0].as_json(), 'sample_qr.json')
mh_response_load = qr.QuestionnaireResponse.create(mh_questionnaire_responses[0], smart)
mh_response_id = mh_response_load["id"]
print(mh_response_id)


531840


In [223]:
print(pa_questionnaire_responses[1].as_json())
print("")
print(mh_questionnaire_responses[0].as_json())

{'identifier': {'system': 'https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/PAQ_J.htm', 'value': '93706'}, 'item': [{'answer': [{'valueInteger': 45}], 'linkId': 'PAD645'}, {'answer': [{'valueInteger': 60}], 'linkId': 'PAD675'}, {'answer': [{'valueInteger': 60}], 'linkId': 'PAD680'}, {'answer': [{'valueInteger': 2}], 'linkId': 'PAQ605'}, {'answer': [{'valueInteger': 2}], 'linkId': 'PAQ620'}, {'answer': [{'valueInteger': 2}], 'linkId': 'PAQ635'}, {'answer': [{'valueInteger': 2}], 'linkId': 'PAQ650'}, {'answer': [{'valueInteger': 1}], 'linkId': 'PAQ665'}, {'answer': [{'valueInteger': 3}], 'linkId': 'PAQ670'}], 'questionnaire': 'Questionnaire/531842', 'status': 'completed', 'resourceType': 'QuestionnaireResponse'}

{'identifier': {'system': 'https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DPQ_J.htm', 'value': '93705'}, 'item': [{'answer': [{'valueInteger': 0}], 'linkId': 'DPQ010'}, {'answer': [{'valueInteger': 0}], 'linkId': 'DPQ020'}, {'answer': [{'valueInteger': 0}], 'linkId': 'DPQ030'}, {'answer':

In [221]:
pa_quest_download.item[1].text

'In a typical week, on how many days {do you/does SP} do vigorous-intensity activities as part of {your/his/her} work?'